[![下载Notebook](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/website-images/master/resource/_static/logo_notebook.png)](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/notebook/master/tutorials/zh_cn/beginner/mindspore_transforms.ipynb)&emsp;[![下载样例代码](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/website-images/master/resource/_static/logo_download_code.png)](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/notebook/master/tutorials/zh_cn/beginner/mindspore_transforms.py)&emsp;[![查看源文件](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/website-images/master/resource/_static/logo_source.png)](https://gitee.com/mindspore/docs/blob/master/tutorials/source_zh_cn/beginner/transforms.ipynb)

[基本介绍](https://www.mindspore.cn/tutorials/zh-CN/master/beginner/introduction.html) || [快速入门](https://www.mindspore.cn/tutorials/zh-CN/master/beginner/quick_start.html) || [张量 Tensor](https://www.mindspore.cn/tutorials/zh-CN/master/beginner/tensor.html) || [数据集 Dataset](https://www.mindspore.cn/tutorials/zh-CN/master/beginner/dataset.html) || **数据变换 Transforms** || [网络构建](https://www.mindspore.cn/tutorials/zh-CN/master/beginner/model.html) || [函数式自动微分](https://www.mindspore.cn/tutorials/zh-CN/master/beginner/autograd.html) || [模型训练](https://www.mindspore.cn/tutorials/zh-CN/master/beginner/train.html) || [保存与加载](https://www.mindspore.cn/tutorials/zh-CN/master/beginner/save_load.html)

# 数据变换 Transforms

通常情况下，直接加载的原始数据并不能直接送入神经网络进行训练，此时我们需要对其进行数据预处理。MindSpore提供不同种类的数据变换（Transforms），配合数据处理Pipeline来实现数据预处理。所有的Transforms均可通过`map`方法传入，实现对指定数据列的处理。

`mindspore.dataset`提供了面向图像、文本、音频等不同数据类型的Transforms，同时也支持使用Lambda函数。下面分别对其进行介绍。

In [1]:
import numpy as np
from PIL import Image
from download import download
from mindspore.dataset import transforms, vision, text
from mindspore.dataset import GeneratorDataset, MnistDataset

## Common Transforms

`mindspore.dataset.transforms`模块支持一系列通用Transforms。这里我们以`Compose`为例，介绍其使用方式。

### Compose

`Compose`接收一个数据增强操作序列，然后将其组合成单个数据增强操作。我们仍基于Mnist数据集呈现Transforms的应用效果。

In [2]:
# Download data from open datasets

url = "https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/" \
      "notebook/datasets/MNIST_Data.zip"
path = download(url, "./", kind="zip", replace=True)

train_dataset = MnistDataset('MNIST_Data/train')


file_sizes: 100%|██████████████████████████| 10.8M/10.8M [00:01<00:00, 9.01MB/s]
Extracting zip file...
Successfully downloaded / unzipped to ./


In [3]:
image, label = next(train_dataset.create_tuple_iterator())
print(image.shape)

(28, 28, 1)


In [4]:
composed = transforms.Compose(
    [
        vision.Rescale(1.0 / 255.0, 0),
        vision.Normalize(mean=(0.1307,), std=(0.3081,)),
        vision.HWC2CHW()
    ]
)

In [5]:
train_dataset = train_dataset.map(composed, 'image')
image, label = next(train_dataset.create_tuple_iterator())
print(image.shape)

(1, 28, 28)


更多通用Transforms详见[mindspore.dataset.transforms](https://www.mindspore.cn/docs/zh-CN/master/api_python/mindspore.dataset.transforms.html)。

## Vision Transforms

`mindspore.dataset.vision`模块提供一系列针对图像数据的Transforms。在Mnist数据处理过程中，使用了`Rescale`、`Normalize`和`HWC2CHW`变换。下面对其进行详述。

### Rescale

`Rescale`变换用于调整图像像素值的大小，包括两个参数：

- rescale：缩放因子。
- shift：平移因子。

图像的每个像素将根据这两个参数进行调整，输出的像素值为$output_{i} = input_{i} * rescale + shift$。

这里我们先使用numpy随机生成一个像素值在\[0, 255\]的图像，将其像素值进行缩放。

In [6]:
random_np = np.random.randint(0, 255, (48, 48), np.uint8)
random_image = Image.fromarray(random_np)
print(random_np)

[[170  10 218 ...  81 128  96]
 [  2 107 146 ... 239 178 165]
 [232 137 235 ... 222 109 216]
 ...
 [193 140  60 ...  72 133 144]
 [232 175  58 ...  55 110  94]
 [152 241 105 ... 187  45  43]]


为了更直观地呈现Transform前后的数据对比，我们使用Transforms的[Eager模式](https://www.mindspore.cn/tutorials/zh-CN/master/advanced/dataset/eager.html)进行演示。首先实例化Transform对象，然后调用对象进行数据处理。

In [7]:
rescale = vision.Rescale(1.0 / 255.0, 0)
rescaled_image = rescale(random_image)
print(rescaled_image)

[[0.6666667  0.03921569 0.854902   ... 0.31764707 0.5019608  0.37647063]
 [0.00784314 0.41960788 0.57254905 ... 0.93725497 0.69803923 0.64705884]
 [0.909804   0.5372549  0.9215687  ... 0.8705883  0.427451   0.8470589 ]
 ...
 [0.7568628  0.54901963 0.23529413 ... 0.28235295 0.52156866 0.5647059 ]
 [0.909804   0.6862745  0.227451   ... 0.21568629 0.43137258 0.36862746]
 [0.59607846 0.9450981  0.41176474 ... 0.73333335 0.1764706  0.16862746]]


可以看到，使用`Rescale`后的每个像素值都进行了缩放。

### Normalize

`Normalize`变换用于对输入图像的归一化，包括三个参数：

- mean：图像每个通道的均值。
- std：图像每个通道的标准差。
- is_hwc：输入图像格式为(height, width, channel)还是(channel, height, width)。

图像的每个通道将根据`mean`和`std`进行调整，计算公式为$output_{c} = \frac{input_{c} - mean_{c}}{std_{c}}$，其中 $c$代表通道索引。

In [8]:
normalize = vision.Normalize(mean=(0.1307,), std=(0.3081,))
normalized_image = normalize(rescaled_image)
print(normalized_image)

[[ 1.7395868  -0.29693064  2.3505423  ...  0.60677403  1.2050011
   0.7976976 ]
 [-0.3987565   0.9377082   1.4341093  ...  2.617835    1.8414128
   1.6759458 ]
 [ 2.5287375   1.3195552   2.5669222  ...  2.4014552   0.9631647
   2.3250859 ]
 ...
 [ 2.0323365   1.3577399   0.33948112 ...  0.49221992  1.2686423
   1.4086528 ]
 [ 2.5287375   1.803228    0.31402466 ...  0.27583995  0.9758929
   0.77224106]
 [ 1.5104787   2.6432917   0.9122518  ...  1.9559668   0.14855757
   0.12310111]]


### HWC2CWH

`HWC2CWH`变换用于转换图像格式。在不同的硬件设备中可能会对(height, width, channel)或(channel, height, width)两种不同格式有针对性优化。MindSpore设置HWC为默认图像格式，在有CWH格式需求时，可使用该变换进行处理。

这里我们先将前文中`normalized_image`处理为HWC格式，然后进行转换。可以看到转换前后的shape发生了变化。

In [9]:
hwc_image = np.expand_dims(normalized_image, -1)
hwc2cwh = vision.HWC2CHW()
chw_image = hwc2cwh(hwc_image)
print(hwc_image.shape, chw_image.shape)

(48, 48, 1) (1, 48, 48)


更多Vision Transforms详见[mindspore.dataset.vision](https://www.mindspore.cn/docs/zh-CN/master/api_python/mindspore.dataset.transforms.html#视觉)。

## Text Transforms

`mindspore.dataset.text`模块提供一系列针对文本数据的Transforms。与图像数据不同，文本数据需要有分词（Tokenize）、构建词表、Token转Index等操作。这里简单介绍其使用方法。

首先我们定义三段文本，作为待处理的数据，并使用`GeneratorDataset`进行加载。

In [10]:
texts = ['Welcome to Beijing']

In [11]:
test_dataset = GeneratorDataset(texts, 'text')

### PythonTokenizer

分词（Tokenize）操作是文本数据的基础处理方法，MindSpore提供多种不同的Tokenizer。这里我们选择基础的`PythonTokenizer`举例，此Tokenizer允许用户自由实现分词策略。随后我们利用`map`操作将此分词器应用到输入的文本中，对其进行分词。

In [12]:
def my_tokenizer(content):
    return content.split()

test_dataset = test_dataset.map(text.PythonTokenizer(my_tokenizer))
print(next(test_dataset.create_tuple_iterator()))

[Tensor(shape=[3], dtype=String, value= ['Welcome', 'to', 'Beijing'])]


### Lookup

`Lookup`为词表映射变换，用来将Token转换为Index。在使用`Lookup`前，需要构造词表，一般可以加载已有的词表，或使用`Vocab`生成词表。这里我们选择使用`Vocab.from_dataset`方法从数据集中生成词表。

In [13]:
vocab = text.Vocab.from_dataset(test_dataset)

获得词表后我们可以使用`vocab`方法查看词表。

In [14]:
print(vocab.vocab())

{'to': 2, 'Beijing': 0, 'Welcome': 1}


生成词表后，可以配合`map`方法进行词表映射变换，将Token转为Index。

In [15]:
test_dataset = test_dataset.map(text.Lookup(vocab))
print(next(test_dataset.create_tuple_iterator()))

[Tensor(shape=[3], dtype=Int32, value= [1, 2, 0])]


更多Text Transforms详见[mindspore.dataset.text](https://www.mindspore.cn/docs/zh-CN/master/api_python/mindspore.dataset.transforms.html#文本)。

## Lambda Transforms

Lambda函数是一种不需要名字、由一个单独表达式组成的匿名函数，表达式会在调用时被求值。Lambda Transforms可以加载任意定义的Lambda函数，提供足够的灵活度。在这里，我们首先使用一个简单的Lambda函数，对输入数据乘2：

In [16]:
test_dataset = GeneratorDataset([1, 2, 3], 'data', shuffle=False)
test_dataset = test_dataset.map(lambda x: x * 2)
print(list(test_dataset.create_tuple_iterator()))

[[Tensor(shape=[], dtype=Int64, value= 2)], [Tensor(shape=[], dtype=Int64, value= 4)], [Tensor(shape=[], dtype=Int64, value= 6)]]


可以看到`map`传入Lambda函数后，迭代获得数据进行了乘2操作。

我们也可以定义较复杂的函数，配合Lambda函数实现复杂数据处理：

In [17]:
def func(x):
    return x * x + 2

test_dataset = test_dataset.map(lambda x: func(x))

In [18]:
print(list(test_dataset.create_tuple_iterator()))

[[Tensor(shape=[], dtype=Int64, value= 6)], [Tensor(shape=[], dtype=Int64, value= 18)], [Tensor(shape=[], dtype=Int64, value= 38)]]
